### Ingestion file "movie_genre"

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Cargar el archivo "movie_genre.json"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType

In [0]:
movie_genre_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("genreId", IntegerType(), True)
])

In [0]:
movie_genre_df = spark.read \
    .schema(movie_genre_schema) \
    .json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")

#### Paso 2 - Añadir y renombrar las columnas

1. Renombrar "movieId" a "movie_id"
2. Renombrar "genreId" a "genre_id"
3. Añadir columna "ingestion_date"
4. Añadir columna "environment"

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
movie_genre_final_df = add_ingestion_date(movie_genre_df) \
    .withColumnRenamed("movieId", "movie_id") \
    .withColumnRenamed("genreId", "genre_id") \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))

#### Paso 3 - Cargar el archivo en Parquet

In [0]:
#overwrite_partition_data(movie_genre_final_df, "movie_silver", "movies_genres", "file_date")
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date'

merge_delta_lake(movie_genre_final_df, 'movie_silver', 'movies_genres', silver_folder_path, merge_condition, 'file_date')

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.movies_genres
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("Success")